In [1]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular import data_setup, engine

In [2]:
from pathlib import Path

image_path = Path('data/pizza_steak_sushi/')
image_path

WindowsPath('data/pizza_steak_sushi')

In [3]:
train_path = image_path / 'train'
test_path = image_path / 'test'

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    normalize
])

In [5]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_path,
                                                                               test_dir=test_path,
                                                                               transform=manual_transform,
                                                                               batch_size=32)

In [6]:
len(train_dataloader), len(test_dataloader)

(8, 3)

In [7]:
class_names

['pizza', 'steak', 'sushi']

In [8]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)

In [9]:
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                                 nn.Linear(out_features=len(class_names), in_features=1280))

In [10]:
from torchinfo import summary

# # Get a summary of the model (uncomment for full output)
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [11]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [12]:
from going_modular.utils import create_writer

example_writer = create_writer('testing_writer', 'efficientnet_b0')

[INFO] Created SummaryWriter, saving to: runs\2024-09-30\testing_writer\efficientnet_b0...


In [13]:
results = engine.train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device='cpu',
                writer=example_writer)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0470 | train_acc: 0.4258 | test_loss: 0.8832 | test_acc: 0.7330
Epoch: 2 | train_loss: 0.8626 | train_acc: 0.6836 | test_loss: 0.7552 | test_acc: 0.8352
Epoch: 3 | train_loss: 0.7245 | train_acc: 0.8828 | test_loss: 0.7246 | test_acc: 0.8040
Epoch: 4 | train_loss: 0.7157 | train_acc: 0.7266 | test_loss: 0.6637 | test_acc: 0.8456
Epoch: 5 | train_loss: 0.6393 | train_acc: 0.8008 | test_loss: 0.6287 | test_acc: 0.8665


In [14]:
train_path_10_percent = image_path / 'train'
test_path = image_path / 'test'

train_path_20_percent = Path('data/pizza_steak_sushi_20_percent') / 'train'
train_path_20_percent

WindowsPath('data/pizza_steak_sushi_20_percent/train')

In [15]:
manual_transform

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [16]:
BATCH_SIZE = 32     
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_path_10_percent,
                                                                                          test_dir=test_path,
                                                                                          transform=manual_transform,
                                                                                          batch_size=BATCH_SIZE)
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_path_20_percent,
                                                                                          test_dir=test_path,
                                                                                          transform=manual_transform,
                                                                                          batch_size=BATCH_SIZE)

In [17]:
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(test_dataloader)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 3 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


In [18]:
OUT_FEATURES = len(class_names)
def create_effnetb0():

    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights)

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES, bias=True)
    )

    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb2():

    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights)

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES, bias=True)
    )

    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

In [19]:
effnetb0 = create_effnetb0()
effnetb2 = create_effnetb2()

[INFO] Created new effnetb0 model.
[INFO] Created new effnetb2 model.


In [20]:
num_epochs = [5, 10]
models = ['effnetb0', 'effnetb2']
train_dataloaders = {'data_10_percent' : train_dataloader_10_percent,
                     'data_20_percent' : train_dataloader_20_percent}

In [21]:
from going_modular.utils import save_model

In [22]:

experiment_number = 0
for dataloader_name, train_dataloader in train_dataloaders.items():

    for epochs in num_epochs:

        for model_name in models:

            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")

            if model_name == 'effnetb0':
                model = create_effnetb0()
            else:
                model = create_effnetb2()

            loss_fn = torch.nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr = 0.001)

            engine.train(model=model,
                                   train_dataloader=train_dataloader,
                                   test_dataloader=test_dataloader,
                                   optimizer=optimizer,
                                   loss_fn=loss_fn,
                                   epochs=epochs,
                                   device='cpu',
                                   writer=create_writer(experiment_name=dataloader_name,
                                                        model_name=model_name,
                                                        extra=f'{epochs}_epochs'))
            
            save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                       target_dir="models",
                       model_name=save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-09-30\data_10_percent\effnetb0\5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0603 | train_acc: 0.4531 | test_loss: 0.8599 | test_acc: 0.7737
Epoch: 2 | train_loss: 0.8492 | train_acc: 0.7109 | test_loss: 0.7185 | test_acc: 0.8665
Epoch: 3 | train_loss: 0.7747 | train_acc: 0.7422 | test_loss: 0.6092 | test_acc: 0.9375
Epoch: 4 | train_loss: 0.7031 | train_acc: 0.7578 | test_loss: 0.6161 | test_acc: 0.8769
Epoch: 5 | train_loss: 0.6057 | train_acc: 0.9023 | test_loss: 0.5462 | test_acc: 0.9072
[INFO] Saving model to: models\07_effnetb0_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: effnetb2
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created new effnetb2 model.
[INFO] Created SummaryWriter, saving to: runs\2024-09-30\data_10_percent\effnetb2\5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0258 | train_acc: 0.4961 | test_loss: 0.9667 | test_acc: 0.5492
Epoch: 2 | train_loss: 0.8426 | train_acc: 0.6719 | test_loss: 0.8600 | test_acc: 0.6420
Epoch: 3 | train_loss: 0.8240 | train_acc: 0.6484 | test_loss: 0.7855 | test_acc: 0.8258
Epoch: 4 | train_loss: 0.6825 | train_acc: 0.8984 | test_loss: 0.7028 | test_acc: 0.8561
Epoch: 5 | train_loss: 0.5771 | train_acc: 0.9219 | test_loss: 0.6824 | test_acc: 0.8665
[INFO] Saving model to: models\07_effnetb2_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 3
[INFO] Model: effnetb0
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 10
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-09-30\data_10_percent\effnetb0\10_epochs...


...

Epoch: 1 | train_loss: 0.9720 | train_acc: 0.5500 | test_loss: 0.8024 | test_acc: 0.8352
Epoch: 2 | train_loss: 0.7104 | train_acc: 0.8042 | test_loss: 0.6422 | test_acc: 0.8759
Epoch: 3 | train_loss: 0.6017 | train_acc: 0.8167 | test_loss: 0.6024 | test_acc: 0.8258
Epoch: 4 | train_loss: 0.4648 | train_acc: 0.9167 | test_loss: 0.5244 | test_acc: 0.8665
Epoch: 5 | train_loss: 0.4565 | train_acc: 0.8583 | test_loss: 0.4850 | test_acc: 0.8466
[INFO] Saving model to: models\07_effnetb2_data_20_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 7
[INFO] Model: effnetb0
[INFO] DataLoader: data_20_percent
[INFO] Number of epochs: 10
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs\2024-09-30\data_20_percent\effnetb0\10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9641 | train_acc: 0.5729 | test_loss: 0.6634 | test_acc: 0.9271
Epoch: 2 | train_loss: 0.7045 | train_acc: 0.8271 | test_loss: 0.5508 | test_acc: 0.8769
Epoch: 3 | train_loss: 0.5585 | train_acc: 0.8792 | test_loss: 0.4781 | test_acc: 0.8977
Epoch: 4 | train_loss: 0.4624 | train_acc: 0.8896 | test_loss: 0.4001 | test_acc: 0.9176
Epoch: 5 | train_loss: 0.4188 | train_acc: 0.8833 | test_loss: 0.3740 | test_acc: 0.9081
Epoch: 6 | train_loss: 0.4228 | train_acc: 0.8625 | test_loss: 0.3591 | test_acc: 0.8873
Epoch: 7 | train_loss: 0.3787 | train_acc: 0.8729 | test_loss: 0.3443 | test_acc: 0.9186
Epoch: 8 | train_loss: 0.3466 | train_acc: 0.9125 | test_loss: 0.3127 | test_acc: 0.9384
Epoch: 9 | train_loss: 0.2969 | train_acc: 0.9167 | test_loss: 0.3231 | test_acc: 0.9081
Epoch: 10 | train_loss: 0.3327 | train_acc: 0.9125 | test_loss: 0.3197 | test_acc: 0.9280
[INFO] Saving model to: models\07_effnetb0_data_20_percent_10_epochs.pth
------------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9701 | train_acc: 0.5667 | test_loss: 0.8035 | test_acc: 0.7746
Epoch: 2 | train_loss: 0.7157 | train_acc: 0.8208 | test_loss: 0.6540 | test_acc: 0.8561
Epoch: 3 | train_loss: 0.5618 | train_acc: 0.8708 | test_loss: 0.5701 | test_acc: 0.8561
Epoch: 4 | train_loss: 0.4946 | train_acc: 0.9229 | test_loss: 0.5477 | test_acc: 0.8665
Epoch: 5 | train_loss: 0.4923 | train_acc: 0.8354 | test_loss: 0.4644 | test_acc: 0.9176
Epoch: 6 | train_loss: 0.3910 | train_acc: 0.9187 | test_loss: 0.4915 | test_acc: 0.8466
Epoch: 7 | train_loss: 0.3724 | train_acc: 0.9229 | test_loss: 0.4339 | test_acc: 0.8977
Epoch: 8 | train_loss: 0.3422 | train_acc: 0.9104 | test_loss: 0.4319 | test_acc: 0.8674
Epoch: 9 | train_loss: 0.3834 | train_acc: 0.8625 | test_loss: 0.3992 | test_acc: 0.8977
Epoch: 10 | train_loss: 0.3272 | train_acc: 0.8708 | test_loss: 0.3627 | test_acc: 0.9384
[INFO] Saving model to: models\07_effnetb2_data_20_percent_10_epochs.pth
------------------------------------